# setup

In [1]:
import torch
import torch.nn as nn
import os

In [2]:
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [3]:
from dataloader import Cifar10

# init

In [4]:
model_label = 'resnet18'
model = torch.hub.load('pytorch/vision:v0.10.0', model_label, pretrained = False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 2
BATCH_SIZE = 128
NUM_WORKERS = 6

Using cache found in /home/josegfer/.cache/torch/hub/pytorch_vision_v0.10.0


In [5]:
trn_ds = Cifar10(split = 'train')
trn_loader = torch.utils.data.DataLoader(trn_ds, batch_size = BATCH_SIZE,
                                          shuffle = True, num_workers = NUM_WORKERS)

val_ds = Cifar10(split = 'val')
val_loader = torch.utils.data.DataLoader(val_ds, batch_size = BATCH_SIZE,
                                         shuffle = False, num_workers = NUM_WORKERS)

In [6]:
import json

from tqdm import tqdm

from utils import plot_log, export

class Runner():
    def __init__(self, device, model, model_label):
        self.device = device
        self.model = model
        self.model_label = model_label
        if not os.path.exists('output/{}'.format(model_label)):
            os.makedirs('output/{}'.format(model_label))
    
    def train(self, epochs, trn_loader, val_loader):
        self.model = self.model.to(self.device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters())

        for epoch in range(epochs):
            print('-- epoch {}'.format(epoch))
            trn_log = self._train_loop(trn_loader, optimizer, criterion)['trn_log']
            val_log = self._eval_loop(val_loader, criterion)['val_log']
            plot_log(self.model_label, trn_log, val_log, epoch)
        export(self.model, self.model_label)

    def _train_loop(self, loader, optimizer, criterion):
        log = []
        self.model.train()
        for batch in tqdm(loader):
            x = batch['image'].to(self.device)
            y = batch['label'].to(self.device)

            yhat = self.model.forward(x)
            loss = criterion(yhat, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            log.append(loss.item())
        return {'trn_log': log}
    
    def _eval_loop(self, loader, criterion):
        log = 0
        self.model.eval()
        with torch.no_grad():
            for batch in tqdm(loader):
                x = batch['image'].to(self.device)
                y = batch['label'].to(self.device)

                yhat = self.model.forward(x)
                loss = criterion(yhat, y)

                log += loss.item()
        return {'val_log': log / len(loader)}
    
    def acc(self, loader):
        num = 0
        den = 0
        self.model.eval()
        with torch.no_grad():
            for batch in tqdm(loader):
                x = batch['image'].to(self.device)
                y = batch['label'].to(self.device)

                yhat = self.model.forward(x)
                _, predicted = torch.max(yhat.data, 1)
                den += y.size(0)
                num += (predicted == y).sum().item()
        log = {'acc': num / den}
        with open('output/{}/accuracy.json'.format(self.model_label), 'w') as file:
            json.dump(log, file)
        return log

In [7]:
runner = Runner(device, model, model_label)

# train

In [8]:
runner.train(EPOCHS, trn_loader, val_loader)

-- epoch 0


  0%|          | 0/391 [00:00<?, ?it/s]/home/josegfer/miniconda3/envs/mirror/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 79/79 [00:11<00:00,  6.80it/s]


-- epoch 1


100%|██████████| 79/79 [00:11<00:00,  6.96it/s]


exporting model


# eval

In [9]:
acc = runner.acc(val_loader)
acc['acc']

100%|██████████| 79/79 [00:11<00:00,  6.59it/s]


0.5466